In [1]:
import torch
import torchvision
from torchvision import models,datasets,transforms
import os
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np

In [2]:
#sos remember to change the directory for the tensors
dir = '../data/dataset_erasing_last_removing_color02-version1'

params = { 'batch_size':16,
           'shuffle':True,
           'num_workers':4 }


transform = transforms.Compose([transforms.Resize(256),
                                transforms.RandomResizedCrop(256), #Augmented
                                transforms.ToTensor(),
                                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

#the training dataset will be from the datase_balanced folder 
train_dataset = datasets.ImageFolder(os.path.join(dir, 'train'),transform = transform )

transform = transforms.Compose([transforms.Resize(256),
                                transforms.CenterCrop(256),
                                transforms.ToTensor(),
                                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])


#change the dir for validationa (and test)
dir = '../data/dataset_split'
val_dataset = datasets.ImageFolder(os.path.join(dir, 'val'),transform = transform )


#data loaders
train_dataloader = torch.utils.data.DataLoader(train_dataset, **params)
val_dataloader = torch.utils.data.DataLoader(val_dataset, **params)

class_names = train_dataset.classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
print('Train dataset = {}\n'.format(len(train_dataset)),'Val dataset = {}'.format(len(val_dataset)))
print('Classes = {}'.format(class_names))

Train dataset = 8832
 Val dataset = 719
Classes = ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']


In [4]:
import torch
import pandas as pd
import csv

def train(model, loss_fn, optimizer, num_epochs=25, save_path='ttrained_realwaste_erasing_version_05_val_densenet.csv'):
    
    best_acc = 0
    results = []  # Store epoch-wise results as lists of arrays

    for epoch in range(num_epochs):
        
        print('Epoch {}'.format(epoch+1))
        
        # Train dataset
        model.train()
        train_loss = 0.0
        train_correct = 0
        size = len(train_dataset)
        for inputs, labels in train_dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()  # Zero the gradients
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = loss_fn(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * inputs.size(0)
            train_correct += torch.sum(preds == labels.data)
            
        train_loss = train_loss / size
        train_acc = train_correct.double() / size
            
        print('Training Loss: {:.4f} Acc: {:.4f}'.format(train_loss, train_acc))
        
        # Validation dataset
        model.eval()
        val_loss = 0.0
        val_correct = 0
        size = len(val_dataset)
        for inputs, labels in val_dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            with torch.no_grad():
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = loss_fn(outputs, labels)
            val_loss += loss.item() * inputs.size(0)
            val_correct += torch.sum(preds == labels.data) 

        val_loss = val_loss / size
        val_acc = val_correct.double() / size    

        print('Validation Loss: {:.4f} Acc: {:.4f}'.format(val_loss, val_acc))    
            
        if val_acc > best_acc:
            best_acc = val_acc

        # Store the results for this epoch as a list of arrays
        results.append([epoch + 1, [train_loss], [train_acc.item()], [val_loss], [val_acc.item()]])
    
    # Save results to CSV
    df = pd.DataFrame(results, columns=['Epoch', 'Train Loss', 'Train Accuracy', 'Validation Loss', 'Validation Accuracy'])
    df.to_csv(save_path, index=False)
    
    print('Best Validation Accuracy: {:.4f}'.format(best_acc))
    print(f'Training results saved to {save_path}')
    
    return model

In [5]:
net = torchvision.models.densenet121(pretrained=True)
for param in net.parameters():
    param.requires_grad = False # freeze all the weights

ft = net.classifier.in_features # final layer of the densenet
net.classifier = nn.Linear(ft, 6) # new layer according to our dataset with weights unfrozen

net = net.to(device)

loss = nn.CrossEntropyLoss()

# only final layer optimized
optimizer = optim.SGD(net.classifier.parameters(), lr=0.0001, momentum=0.9) 

C:\Users\vidia\AppData\Roaming\Python\Python312\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\vidia\AppData\Roaming\Python\Python312\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [6]:
net = train(net,loss,optimizer,num_epochs=100)



Epoch 1
Training Loss: 1.6184 Acc: 0.3576
Validation Loss: 1.3187 Acc: 0.5800
Epoch 2
Training Loss: 1.3631 Acc: 0.5322
Validation Loss: 1.0851 Acc: 0.6718
Epoch 3
Training Loss: 1.2438 Acc: 0.5728
Validation Loss: 0.9751 Acc: 0.7149
Epoch 4
Training Loss: 1.1704 Acc: 0.6011
Validation Loss: 0.9038 Acc: 0.7246
Epoch 5
Training Loss: 1.1171 Acc: 0.6170
Validation Loss: 0.8631 Acc: 0.7107
Epoch 6
Training Loss: 1.0821 Acc: 0.6303
Validation Loss: 0.8273 Acc: 0.7177
Epoch 7
Training Loss: 1.0476 Acc: 0.6392
Validation Loss: 0.8079 Acc: 0.7260
Epoch 8
Training Loss: 1.0322 Acc: 0.6381
Validation Loss: 0.7796 Acc: 0.7344
Epoch 9
Training Loss: 1.0084 Acc: 0.6487
Validation Loss: 0.7620 Acc: 0.7357
Epoch 10
Training Loss: 0.9965 Acc: 0.6541
Validation Loss: 0.7445 Acc: 0.7469
Epoch 11
Training Loss: 0.9857 Acc: 0.6547
Validation Loss: 0.7486 Acc: 0.7344
Epoch 12
Training Loss: 0.9650 Acc: 0.6634
Validation Loss: 0.7184 Acc: 0.7552
Epoch 13
Training Loss: 0.9572 Acc: 0.6662
Validation Loss: 0

Saving the parameters of the desnet model after training!

In [7]:
import torch

torch.save(net.state_dict(), "trained_realwaste_erasing05_models_val_densenet.pth")  # Save model weights
print("Model saved as trained_densenet.pth")


Model saved as trained_densenet.pth
